# Reformatting .pkl to panel data

Expect inputs in .pkl:
- combined jstor + scopus metadata
- author names and affiliations data
- references data
- tables, figures and equations data


Output:
- flattened versions

REC:
- 013, 023


In [1]:
import pandas as pd
# from unidecode import unidecode
from datetime import date
import numpy as np
import time
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import pickle
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.pipeline import make_pipeline



scaler = StandardScaler()


pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_rows', None)

In [2]:
base_path="/Users/sijiawu/Work/Thesis/Data/Affiliations/"
data_base_path="/Users/sijiawu/Work/Thesis/Data/"
nets_path="/Users/sijiawu/Work/80YearsEconomicResearch/032_auth_graph_gen/networks/"
pdf_base_path="/Users/sijiawu/Dropbox/80YearsEconomicResearch/Data/0_PDF/"

In [3]:
proc_auths_all = pd.read_pickle(base_path+"proc_auth_aff_flat.pkl")
aff_sub=pd.read_pickle(base_path+"affiliations_combined_sub.pkl")
j_data=pd.read_pickle(data_base_path+"Combined/011_merged_proc_scopus_inception_2020_w_counts.pkl")
all_refs=pd.read_excel('../031_proc_refs_full_set/refs_1940_2020.xlsx')
relevant=pd.read_excel('../031_proc_refs_full_set/refs_1940_2020_top5.xlsx')

In [4]:
def compute_jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

In [5]:
j_data["id"]=j_data["URL"].str.split("/").str[-1]
relevant["id_o"]=relevant["id_o"].astype(str)
relevant["year_o"]=relevant["year_o"].astype(int)
proc_auths_all["id_o"]=proc_auths_all["url"].str.split("/").str[-1]
proc_auths_all["a1_order_str"]=proc_auths_all["a1_order"].astype(str)
proc_auths_all["a2_order_str"]=proc_auths_all["a2_order"].astype(str)

relevant_sub=relevant[["ref_ord", "id_o", "year_o","match_id"]]

In [6]:
for i in proc_auths_all['id_o'].unique():
    if "." in i:
        print(i)

In [7]:
j_data["page_start"]=j_data["pages"].str.split('-').str[0]
j_data["page_end"]=j_data["pages"].str.split('-').str[-1]

In [8]:
ex_content=['MISC', 'Errata','Discussion', 'Review', 'Review2']
content=['Article', 'Comment', 'Reply', 'Rejoinder']

In [9]:
j_data.content_type.unique()

array(['Article', 'MISC', 'Comment', 'Reply', 'Errata', 'Rejoinder',
       'Discussion', 'Review', 'Review2'], dtype=object)

In [10]:
j_data[['pages',
       'year', 'ISSN', 'abstract', 'URL', 'publisher', 'content_type', 'type',
       'jid', 'author_split', 'urldate', 'reviewed-author', 'uploaded',
       'title_10', 'URL_og', 'number_og', 'title_og', 'author_og', 'pages_og',]].head()

,pages,year,ISSN,abstract,URL,publisher,content_type,type,jid,author_split,urldate,reviewed-author,uploaded,title_10,URL_og,number_og,title_og,author_og,pages_og
0,1363-1398,2020,0033-5533,"News reports and communication are inherently constrained by space, time, and attention. As a re...",https://doi.org/10.1093/qje/qjaa012,Oxford University Press,Article,N,qje,"['Enke, Benjamin']",NaN,NaN,NaN,NaN,https://doi.org/10.1093/qje/qjaa012,3,What You See Is All There Is*,"Enke, Benjamin",1363-1398
1,nan,2020,"00028282, 19447981",NaN,https://www.jstor.org/stable/26966477,American Economic Association,MISC,N,aer,NaN,2023-09-04 00:00:00,NaN,1.0,NaN,https://www.jstor.org/stable/26966477,12,Front Matter,NaN,NaN
2,3705-3747,2020,"00028282, 19447981",African agricultural markets are characterized by low farmer revenues and high consumer food pri...,https://www.jstor.org/stable/26966478,American Economic Association,Article,N,aer,"['Lauren Falcao Bergquist', 'Michael Dinerstein']",2023-09-04 00:00:00,NaN,1.0,NaN,https://www.jstor.org/stable/26966478,12,Competition and Entry in Agricultural Markets: Experimental Evidence from Kenya,Lauren Falcao Bergquist and Michael Dinerstein,3705-3747
3,3748-3785,2020,"00028282, 19447981",We present a new equilibrium search model where consumers initially search among discount opport...,https://www.jstor.org/stable/26966479,American Economic Association,Article,N,aer,"['Dominic Coey', 'Bradley J. Larsen', 'Brennan C. Platt']",2023-09-04 00:00:00,NaN,1.0,NaN,https://www.jstor.org/stable/26966479,12,Discounts and Deadlines in Consumer Search,Dominic Coey and Bradley J. Larsen and Brennan C. Platt,3748-3785
4,3786-3816,2020,"00028282, 19447981",We formalize the argument that political disagreements can be traced to a “clash of narratives.”...,https://www.jstor.org/stable/26966480,American Economic Association,Article,N,aer,"['Kfir Eliaz', 'Ran Spiegler']",2023-09-04 00:00:00,NaN,1.0,NaN,https://www.jstor.org/stable/26966480,12,A Model of Competing Narratives,Kfir Eliaz and Ran Spiegler,3786-3816


In [11]:
proc_auths_all.shape #46436

(46436, 18)

In [12]:
auth_counts=proc_auths_all["id_o"].value_counts().reset_index()
auth_page_count=auth_counts.merge(j_data[["id","page_count"]], left_on="id_o", right_on="id", how="left").drop_duplicates().drop(columns=["id"])
auth_page_count["points"]=auth_page_count["page_count"]/(auth_page_count["count"]+1)

In [13]:
proc_auths_all=proc_auths_all.merge(auth_page_count, on="id_o", how="left")

In [14]:
auth_point_years=proc_auths_all[["a2_order_str", "year", "points"]].groupby(['year', 'a2_order_str'])['points'].sum().reset_index()

In [15]:
proc_auths_all["c_count"]=proc_auths_all["count"]-1

In [16]:
def lag_count(lag):
    auth_lags=[]
    auth_counts=[]
    paper_counts=[]
    for i in range(1940,2021,1):
        authpoint=auth_point_years[(auth_point_years["year"]<=i)&(auth_point_years["year"]>i-lag)]
        tem=authpoint.groupby(['a2_order_str'])['points'].sum().reset_index()
        tem=tem.rename(columns={"points":"points"+str(lag)})
        tem["year"+str(lag)]=i
        auth_lags.append(tem)

        authcount=proc_auths_all[(proc_auths_all["year"]<=i)&(proc_auths_all["year"]>i-lag)]
        tem=authcount[["a2_order_str", "year", "c_count"]].groupby(['a2_order_str'])['c_count'].sum().reset_index()
        tem=tem.rename(columns={"c_count":"c_counts"+str(lag)})
        tem["year"+str(lag)]=i
        auth_counts.append(tem)

        tem=authcount[["a2_order_str", "year", "c_count"]].groupby(['a2_order_str'])['c_count'].count().reset_index()
        tem=tem.rename(columns={"c_count":"p_counts"+str(lag)})
        tem["year"+str(lag)]=i
        paper_counts.append(tem)

    return (pd.concat(auth_lags) , pd.concat(auth_counts), pd.concat(paper_counts))

In [17]:
auth_lag_10_df, auth_c_10_df, auth_p_10_df = lag_count(10)
auth_lag_5_df, auth_c_5_df, auth_p_5_df = lag_count(5)
auth_lag_20_df, auth_c_20_df, auth_p_20_df = lag_count(20)

In [18]:
auth_lag_5_df.head()

,a2_order_str,points5,year5
0,10,5.5,1940
1,10059,7.0,1940
2,10065,1.5,1940
3,10119,7.0,1940
4,10273,5.5,1940


In [19]:
auth_comps_10=auth_lag_10_df.merge(auth_c_10_df, left_on=["a2_order_str", "year10"], right_on=["a2_order_str", "year10"]).merge(auth_p_10_df, left_on=["a2_order_str", "year10"], right_on=["a2_order_str", "year10"])
auth_comps_20=auth_lag_20_df.merge(auth_c_20_df, left_on=["a2_order_str", "year20"], right_on=["a2_order_str", "year20"]).merge(auth_p_20_df, left_on=["a2_order_str", "year20"], right_on=["a2_order_str", "year20"])
auth_comps_5=auth_lag_5_df.merge(auth_c_5_df, left_on=["a2_order_str", "year5"], right_on=["a2_order_str", "year5"]).merge(auth_p_5_df, left_on=["a2_order_str", "year5"], right_on=["a2_order_str", "year5"])

In [20]:
proc_auths_all.head()

,auth_ord,a1,a2,a3,last,affs,year,content_type,jid,url,...,a3_order,fl,a1_tk_count,id_o,a1_order_str,a2_order_str,count,page_count,points,c_count
0,0,benjamin enke,b. enke,b. enke,enke,"{harvard university, national bureau of economic research - nber}",2020,Article,qje,https://doi.org/10.1093/qje/qjaa012,...,7363,benjamin enke,2,qjaa012,2982,8360,1,35.0,17.500000,0
1,0,lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,Article,aer,https://www.jstor.org/stable/26966478,...,1972,lauren bergquist,3,26966478,13024,7893,2,43.0,14.333333,1
2,1,michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,Article,aer,https://www.jstor.org/stable/26966478,...,6133,michael dinerstein,2,26966478,14901,6936,2,43.0,14.333333,1
3,0,dominic coey,d. coey,d. coey,coey,{facebook},2020,Article,aer,https://www.jstor.org/stable/26966479,...,3164,dominic coey,2,26966479,15321,3640,3,38.0,9.500000,2
4,1,bradley j. larsen,b. j. larsen,b. larsen,larsen,"{national bureau of economic research - nber, stanford university}",2020,Article,aer,https://www.jstor.org/stable/26966479,...,752,bradley larsen,3,26966479,7031,13058,3,38.0,9.500000,2


In [21]:
a1_lags_points=auth_comps_20.merge(auth_comps_10, left_on=["a2_order_str", "year20"], right_on=["a2_order_str", "year10"], how ="left").merge(auth_comps_5, left_on=["a2_order_str", "year20"], right_on=["a2_order_str", "year5"], how ="left").drop(columns=["year10", "year5"]).rename(columns={"year20":"year"})

In [22]:
a1_lags_points.head()

,a2_order_str,points20,year,c_counts20,p_counts20,points10,c_counts10,p_counts10,points5,c_counts5,p_counts5
0,10,5.5,1940,0,1,5.5,0.0,1.0,5.5,0.0,1.0
1,10059,7.0,1940,0,1,7.0,0.0,1.0,7.0,0.0,1.0
2,10065,1.5,1940,0,1,1.5,0.0,1.0,1.5,0.0,1.0
3,10119,7.0,1940,0,1,7.0,0.0,1.0,7.0,0.0,1.0
4,10273,5.5,1940,0,1,5.5,0.0,1.0,5.5,0.0,1.0


In [23]:
def build_coauthorship_network(collabs):
    G = nx.Graph()
    G.add_nodes_from(list(collabs.index))
    unique_pairs = list(combinations(collabs.index, 2))

    for i in unique_pairs:
        if collabs.loc[i[0],i[1]]!=0:
            G.add_edge(i[0], i[1], weight=collabs.loc[i[0],i[1]])
    return G

def get_network_features(G, author1, author2):
    try:
        distance = nx.shortest_path_length(G, source=author1, target=author2)
    except nx.NetworkXNoPath:
        distance = np.inf  # No path exists

    num_paths = len(list(nx.all_shortest_paths(G, source=author1, target=author2))) if distance != np.inf else 0
    return distance, num_paths

def compute_cosine_similarity(matrix):
    m_array = matrix.values
    cosine_sim = cosine_similarity(m_array)
    authors = matrix.index
    cosine_sim_df = pd.DataFrame(cosine_sim, index=authors, columns=authors)
    return cosine_sim_df

def cit_matrix(aff_auths, order, cit_data):
    # Merge to get citations at author level
    print(cit_data.shape)
    df_citations = cit_data.merge(aff_auths[[order, "id_o"]], on="id_o")
    print(df_citations.shape)
    matrix = df_citations.pivot_table(
        index=order, columns="match_id", aggfunc="size", fill_value=0
    )
    co_simm=compute_cosine_similarity(matrix)
    return {"matrix":matrix,"sim_matrix":co_simm}

def collab_matrix(aff_auths, order):
    authors = aff_auths[order].unique()
    author_index = {author: idx for idx, author in enumerate(authors)}
    matrix_size = len(authors)
    collab_matrix = np.zeros((matrix_size, matrix_size), dtype=int)
    grouped_papers = aff_auths.groupby("url")[order].apply(list)

    for authors_list in grouped_papers:
        for author1, author2 in combinations(authors_list, 2):
            idx1, idx2 = author_index[author1], author_index[author2]
            collab_matrix[idx1, idx2] += 1
            collab_matrix[idx2, idx1] += 1  

    collaboration_matrix = pd.DataFrame(collab_matrix, index=authors, columns=authors)
    
    return collaboration_matrix
    
def reduce_affs_jacc_sim(aff_auths, order, cits):
    collabs=collab_matrix(aff_auths, order)
    # o={}
    # for i in aff_auths.index:
    #     if aff_auths.loc[i,order] in o.keys():
    #         o[aff_auths.loc[i,order]].update(aff_auths.loc[i,"affs"])  # Merge sets
    #     else:
    #         o[aff_auths.loc[i,order]] = aff_auths.loc[i,"affs"]

    # Group by the "order" column and merge sets efficiently
    o = aff_auths.groupby(order)["affs"].agg(set.union).to_dict()

    cit_mat=cit_matrix(aff_auths, order, cits)
    
    o_proc=[]
    pairs = list(combinations(list(o.keys()), 2))
    # print(cit_mat['sim_matrix'].head())
    
    G_t = build_coauthorship_network(collabs)
    all_shortest_paths = dict(nx.all_pairs_shortest_path_length(G_t))  # Dictionary of shortest paths
    
    # Convert to DataFrame for fast lookups
    shortest_paths_df = pd.DataFrame(
        [(src, tgt, dist) for src, targets in all_shortest_paths.items() for tgt, dist in targets.items()],
        columns=["pair_1", "pair_2", "distance"]
    )

    print(shortest_paths_df.columns)
    # Compute number of shortest paths
    num_paths_dict = {
        (a, b): len(list(nx.all_shortest_paths(G_t, a, b))) if b in all_shortest_paths[a] else 0
        for a, targets in all_shortest_paths.items()
        for b in targets
    }

    # Add num_paths to DataFrame
    shortest_paths_df["num_paths"] = shortest_paths_df.apply(lambda row: num_paths_dict.get((row["pair_1"], row["pair_2"]), 0), axis=1)

    # Convert pairs list to DataFrame and merge precomputed network data
    # pairs_df = pd.DataFrame(pairs, columns=["pair_1", "pair_2"])
    # pairs_df = pairs_df.merge(shortest_paths_df, on=["pair_1", "pair_2"], how="left").fillna({"distance": np.inf, "num_paths": 0})



    for i in pairs:
        # distance, num_paths = get_network_features(G_t, i[0], i[1])
        # network_proximity = 1 / distance if distance != np.inf else 0
        # log_num_paths = np.log(num_paths + 1)  # Avoid log(0)
        temp={ 
                       "pair_1":i[0], 
                       "pair_2":i[1],
                       "aff_jacc_sim":compute_jaccard_similarity(o[i[0]],o[i[1]]),
                       "collabs": collabs.loc[i[0],i[1]],
                    #    "distance": distance,
                    #    "network_proximity": network_proximity,
                    #    "log_num_paths":log_num_paths
        }
        
        
        if (i[0] in cit_mat["sim_matrix"].index) & (i[1] in cit_mat["sim_matrix"].index):
            temp["cit_cos_sim"]=cit_mat["sim_matrix"].loc[i[0],i[1]]
        else:
            temp["cit_cos_sim"]=0
        o_proc.append(temp)

    combo=pd.DataFrame(o_proc).merge(shortest_paths_df, on=["pair_1", "pair_2"], how="left").fillna({"distance": np.inf, "num_paths": 0})
    
    o_flat=[]
    for i in o.keys():
        o_flat.append({"order": i,"affs": o[i]})

    
    return {"sims_pairs":combo, "aff_sets":o, "aff_flat_sets": pd.DataFrame(o_flat),  "collabs": collabs, "cit":cit_mat, "auth_net": G_t}



def data_prep(t, duration, order):
    print(t)
    aff_t=proc_auths_all[(proc_auths_all['year']<t+duration)&(proc_auths_all['year']>=t)].reset_index(drop=True)
    cit_t=relevant_sub[(relevant_sub["year_o"]>=t)&(relevant_sub["year_o"]<t+duration)].reset_index(drop=True)
    j_t=j_data[(j_data["year"]>=t)&(j_data["year"]<t+duration)][['id', "title", "jid", "year", "content_type"]].reset_index(drop=True)
    a=time.time()
    jaccs=reduce_affs_jacc_sim(aff_t, order, cit_t)
    jaccs["j_data_t"]=j_t
    b=time.time()
    print(b-a)
    # print()
    return jaccs


In [24]:
# jacc_sims_10={}

# for i in range(1940,2020,10):
#     jacc_sims_10[i]=data_prep(i,10, 'a1_order')

# jacc_sims_20={}

# for i in range(1940,2020,20):
#     jacc_sims_20[i]=data_prep(i,20, 'a1_order')

In [657]:
for i in proc_auths_all.index:
    if len(proc_auths_all.loc[i,'a1'].split(" ")[0])==1:
        print(str(proc_auths_all.loc[i,'a1'])+" "+str(proc_auths_all.loc[i,'year'])+" "+str(proc_auths_all.loc[i,'id_o']))
    if len(proc_auths_all.loc[i,'a1'].split(" "))>2:
        if len(proc_auths_all.loc[i,'a1'].split(" ")[1])==1:
            print(str(proc_auths_all.loc[i,'a1'])+" "+str(proc_auths_all.loc[i,'year'])+" "+str(proc_auths_all.loc[i,'id_o']))

# someone fucked up. But it wasn't me.

In [658]:
def pickle_it(filepath, data):
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

def get_pickle(filepath):
    data=None
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    return data

In [25]:

# Load dataset (Assuming it has 'Paper_ID', 'Author', and 'Year')
df = proc_auths_all[["id_o","a2","last", "a2_order_str", "year"]].rename(columns={'id_o': 'Paper_ID', 'a2_order_str': 'Author', "year":"Year"})
# Step 1: Expand multi-author papers into pairwise relationships
df_expanded = df.groupby(["Paper_ID", "Year"])['Author'].apply(
    lambda x: list(combinations(x, 2)) if len(x) > 1 else [(x.iloc[0], None)]
).explode()

# Convert tuples to separate columns
df_expanded = pd.DataFrame(df_expanded.tolist(), index=df_expanded.index, columns=["Author", "Coauthor"]).reset_index()

# Keep necessary columns
df_expanded = df_expanded[["Author", "Coauthor", "Year"]]
df_expanded_alt=df_expanded[["Author", "Coauthor", "Year"]].fillna(-1)

# Step 2: Count occurrences of coauthorship per year
df_expanded["Coauthorship_Count"] = df_expanded.groupby(["Author", "Coauthor", "Year"])['Year'].transform('count')
df_expanded_alt["Coauthorship_Count"] = df_expanded_alt.groupby(["Author", "Coauthor", "Year"])['Year'].transform('count')


# Determine first and last publication year for each author
author_first_pub = df.groupby("Author")["Year"].min()
author_last_pub = df.groupby("Author")["Year"].max()

In [26]:
# Step 3: Define year range
start_year = 1949
end_year = 2020
window_10=10
window_5=5
window_20=20

In [27]:

def cent_measures(G):
    max_eig=max(nx.adjacency_spectrum(G))
    complex_num=max_eig
    float_value = float(complex_num.real)
    # Check if imaginary part is effectively zero
    # has_imaginary_part = not np.isclose(complex_num.imag, 0)
    # components = list(nx.connected_components(G))
    # print(f"Network has {len(components)} disconnected components")
    # Convert to float (only if it's safe to do so)
    # if has_imaginary_part:
        # print(f"Warning: Number has imaginary component: {complex_num.imag}")
        # Handle appropriately - either use abs() or raise error
    # else:
        # float_value = float(complex_num.real)
        # print(f"Converted to float: {float_value}")
    centrality_measures = {
        "degree": nx.degree_centrality(G),
        "betweenness": nx.betweenness_centrality(G),
        "closeness": nx.closeness_centrality(G),
        "eigenvector": nx.eigenvector_centrality(G),
        "pagerank": nx.pagerank(G),
        # "katz": nx.katz_centrality(G, alpha=1/(max_eig+1), max_iter=100000),
        'katz':nx.katz_centrality_numpy(G, alpha=1/(float_value+1)),
        "harmonic": nx.harmonic_centrality(G),
    }
    return centrality_measures



In [28]:
df_expanded_alt.head()

,Author,Coauthor,Year,Coauthorship_Count
0,3410,-1,1998,1
1,5603,3601,1998,1
2,417,-1,1998,1
3,3927,-1,1998,1
4,12686,3711,1998,1


In [29]:
df_expanded.head()

,Author,Coauthor,Year,Coauthorship_Count
0,3410,None,1998,NaN
1,5603,3601,1998,1.0
2,417,None,1998,NaN
3,3927,None,1998,NaN
4,12686,3711,1998,1.0


In [52]:
window=window_10
start_year=1949
# Step 4: Iterate over each year t
for t in range(start_year, end_year + 1):
    w=window-1
    net_dist=[]
    a=time.time()
    print(str(t) + " to "+str(t-w))
    min_year = t - w  # Define 10-year rolling window
    max_year = t
    
    # Filter data for the current 10-year period
    df_window = df_expanded_alt[(df_expanded_alt["Year"] >= min_year) & (df_expanded_alt["Year"] <= max_year)]
    
    # Build coauthorship network
    G = nx.Graph()
    for _, row in df_window.iterrows():
        if row["Coauthor"]==-1:
            G.add_node(row["Author"], weight=row["Coauthorship_Count"])
        else:
            G.add_edge(row["Author"], row["Coauthor"], weight=row["Coauthorship_Count"])
    
    nx.write_gexf(G, nets_path+str(t)+"-"+str(min_year)+"_10Y_a2.gexf", encoding='utf-8')
    b=time.time()
    print(b-a)



1949 to 1940
0.21474480628967285
1950 to 1941
0.04842114448547363
1951 to 1942
0.04773092269897461
1952 to 1943
0.04663205146789551
1953 to 1944
0.046452999114990234
1954 to 1945
0.046221017837524414
1955 to 1946
0.048317909240722656
1956 to 1947
0.04652595520019531
1957 to 1948
0.043540000915527344
1958 to 1949
0.04263806343078613
1959 to 1950
0.043035030364990234
1960 to 1951
0.04535079002380371
1961 to 1952
0.5432438850402832
1962 to 1953
0.045793771743774414
1963 to 1954
0.045552968978881836
1964 to 1955
0.04523587226867676
1965 to 1956
0.046955108642578125
1966 to 1957
0.0486299991607666
1967 to 1958
0.05214214324951172
1968 to 1959
0.05503201484680176
1969 to 1960
0.05830788612365723
1970 to 1961
0.06157183647155762
1971 to 1962
0.07293701171875
1972 to 1963
0.07352495193481445
1973 to 1964
0.07466697692871094
1974 to 1965
0.08112716674804688
1975 to 1966
0.08442902565002441
1976 to 1967
0.08941984176635742
1977 to 1968
0.0944972038269043
1978 to 1969
0.09316086769104004
1979 to 

In [31]:
nets_path

'/Users/sijiawu/Work/80YearsEconomicResearch/032_auth_graph_gen/networks/'

In [32]:
# nx simple paths usage takes way too long

def count_paths_with_exact_length(G, source, target, length):
    return sum(1 for path in nx.all_simple_paths(G, source, target, cutoff=length) 
               if len(path) - 1 == length)


def find_kth_shortest_path(G, k=2):
    results = defaultdict(dict)

    all_paths = [(len(path)-1, path) for path in nx.all_simple_paths(G, source, target)] #add cutoff
    all_paths.sort()


    # Group paths by length
    path_groups = {}
    for length, path in all_paths:
        if length not in path_groups:
            path_groups[length] = []
        path_groups[length].append(path)
    
    # Get distinct lengths and sort them
    lengths = sorted(path_groups.keys())
    
    # If we have at least k different lengths, return one path from the kth group
    if len(lengths) >= k:
        kth_shortest_length = lengths[k-1]
        kth_shortest_path = path_groups[kth_shortest_length][0]  # Get one path of this length
        results[source][target] = (kth_shortest_path, "Success")
    else:
        results[source][target] = ([], f"Only {len(lengths)} path lengths found")

    return results

def find_kth_shortest_paths_all_pairs(G, k=2, limit=5):
    """
    Find the kth shortest path for each pair of nodes in graph G.
    
    Parameters:
    -----------
    G : networkx.Graph
        The input graph
    k : int
        Which shortest path to find (1 for shortest, 2 for second shortest, etc.)
    
    Returns:
    --------
    dict of dict
        A nested dictionary where result[source][target] contains a tuple of
        (path, status) where path is the kth shortest path and status explains
        any issues if the path doesn't exist.
    """
    results = defaultdict(dict)
    
    for source in G.nodes():
        for target in G.nodes():
            if source == target:
                # Skip self-loops
                results[source][target] = ([], "Same node")
                continue

            # Check if there's a path between source and target
            if not nx.has_path(G, source, target):
                results[source][target] = ([], "No path exists")
                continue
            
            try:
                # Get all simple paths and their lengths
                all_paths = [(len(path)-1, path) for path in nx.all_simple_paths(G, source, target, cutoff=limit)]
                
                # Sort paths by length
                all_paths.sort()
                
                # Group paths by length
                path_groups = {}
                for length, path in all_paths:
                    if length not in path_groups:
                        path_groups[length] = []
                    path_groups[length].append(path)
                
                # Get distinct lengths and sort them
                lengths = sorted(path_groups.keys())
                
                # If we have at least k different lengths, return one path from the kth group
                if len(lengths) >= k:
                    kth_shortest_length = lengths[k-1]
                    kth_shortest_path = path_groups[kth_shortest_length][0]  # Get one path of this length
                    results[source][target] = (kth_shortest_path, "Success")
                else:
                    results[source][target] = ([], f"Only {len(lengths)} path lengths found")
            except Exception as e:
                results[source][target] = ([], f"Error: {e}")
    
    return results


In [33]:
# alt count simple paths up to a point, 

def count_simple_paths_of_length(G, source, target, length):
    """
    Counts the number of simple paths of exact length between source and target.
    
    Parameters:
    -----------
    G : NetworkX graph
    source : node
        Starting node
    target : node
        Ending node
    length : int
        Desired path length (number of edges)
        
    Returns:
    --------
    int
        Number of simple paths with exactly the specified length
    """
    # Counter to keep track of the number of paths
    path_count = [0]  # Using list for mutable reference in nested function
    
    # Keep track of visited nodes to ensure simple paths
    visited = {source: True}
    
    def dfs(current, steps_remaining):
        # If we've reached the target with the exact length
        if current == target and steps_remaining == 0:
            path_count[0] += 1
            return
        
        # If we've used all steps but haven't reached target, or reached target too early
        if steps_remaining == 0 or current == target:
            return
            
        # Continue DFS
        for neighbor in G.neighbors(current):
            if neighbor not in visited:
                visited[neighbor] = True
                dfs(neighbor, steps_remaining - 1)
                del visited[neighbor]  # Backtrack
    
    # Start DFS from source
    dfs(source, length)
    return path_count[0]

In [35]:
combo=author_first_pub.reset_index().rename(columns={"Year": "First_Year"}).merge(author_last_pub.reset_index().rename(columns={"Year": "Last_Year"}), on="Author").merge(df[["Author","a2","last"]].groupby("Author").first().reset_index(), on="Author")
combo.head()

,Author,First_Year,Last_Year,a2,last
0,0,2019,2019,j. poulsen,poulsen
1,1,2007,2016,r. mcmillan,mcmillan
2,10,1940,1950,w. miller,miller
3,100,1962,1962,r. c. kao,kao
4,1000,1976,1980,e. ratledge,ratledge


In [36]:
df_ex_na=df_expanded[df_expanded["Coauthor"].isna()==False].reset_index(drop=True)
df_ex_na["Ai"]=df_ex_na["Author"].astype(int)
df_ex_na["Ci"]=df_ex_na["Coauthor"].astype(int)
df_ex_na['Author'] = df_ex_na[['Ai', 'Ci']].min(axis=1).astype(str)
df_ex_na['Coauthor'] = df_ex_na[['Ai', 'Ci']].max(axis=1).astype(str)
df_ex_na.shape

(23582, 6)

In [37]:
collabs=df_ex_na[["Author","Coauthor"]].groupby(["Author","Coauthor"]).first().reset_index()
collabs_merge=collabs.merge(combo, on="Author").merge(combo.rename(columns={"Author":"Coauthor","First_Year":"First_Year_C", "Last_Year":"Last_Year_C", "a1":"a1_C", "last":"last_C"}), on="Coauthor")
collabs_merge.shape

(18719, 10)

In [38]:
collabs_merge.head()

,Author,Coauthor,First_Year,Last_Year,a2_x,last,First_Year_C,Last_Year_C,a2_y,last_C
0,0,2849,2019,2019,j. poulsen,poulsen,2014,2020,j. hjort,hjort
1,1,11519,2007,2016,r. mcmillan,mcmillan,2007,2018,p. bayer,bayer
2,1,16070,2007,2016,r. mcmillan,mcmillan,2007,2012,f. ferreira,ferreira
3,1,2820,2007,2016,r. mcmillan,mcmillan,2002,2016,c. timmins,timmins
4,1,735,2007,2016,r. mcmillan,mcmillan,2015,2016,a. murphy,murphy


In [39]:
df_yijt = {}
a=0
for x in collabs_merge.index:
    i=collabs_merge.loc[x,'Author']
    j=collabs_merge.loc[x,'Coauthor']
    ti_0=collabs_merge.loc[x,'First_Year']
    tj_0=collabs_merge.loc[x,'First_Year_C']
    tij_0=max(ti_0,tj_0)
    ti_2=collabs_merge.loc[x,'Last_Year']
    tj_2=collabs_merge.loc[x,'Last_Year_C']
    tij_2=min(ti_2,tj_2)
    collab_ij=df_ex_na[(df_ex_na["Author"]==i)&(df_ex_na["Coauthor"]==j)]
    collab_years=collab_ij["Year"].to_list()
    # if len(collab_ij["Year"].unique())!=collab_ij.shape[0]:
    #     # print(x)
    #     a+=1
    tij_1=min(collab_ij["Year"])

    for y in range(tij_0, tij_2+1):
        if y not in collab_years:
            df_yijt[a]={"Author":i, "Coauthor":j, "tij":y, "ytij":0, "tij_1":tij_1, "tij_2":tij_2, "tij_0":tij_0, 
                        # "Paper_ID":""
                        }
            a+=1
            # print(y)
    for y in collab_ij.index:
        # print(" "+str(collab_ij.loc[y,"Year"]))
        df_yijt[a]={"Author":i, "Coauthor":j, "tij":collab_ij.loc[y,"Year"], "ytij":1, "tij_1":tij_1, "tij_2":tij_2, "tij_0":tij_0, 
                    # "Paper_ID":collab_ij.loc[y,"Paper_ID"]
                    }
        a+=1



In [40]:
df_y=pd.DataFrame(df_yijt).transpose()

In [41]:
df_y.head()

,Author,Coauthor,tij,ytij,tij_1,tij_2,tij_0
0,0,2849,2019,1,2019,2019,2019
1,1,11519,2008,0,2007,2016,2007
2,1,11519,2009,0,2007,2016,2007
3,1,11519,2010,0,2007,2016,2007
4,1,11519,2011,0,2007,2016,2007


In [42]:
df_y.shape

(118608, 7)

In [43]:
df_y[df_y["tij_0"]==df_y["tij_1"]].shape

(43331, 7)

In [44]:
df_y[df_y["tij"]<=df_y["tij_1"]].shape

(59558, 7)

In [45]:
nms=["i","j","distance", "total_paths", "avg_path_length", "pij_t", "shortest_path_count"]

In [46]:
def count_second_shortest_paths(G, source, target):
    """
    Find the count of second shortest simple paths between source and target.
    
    Parameters:
    -----------
    G : NetworkX graph
    source : node
        Starting node
    target : node
        Ending node
        
    Returns:
    --------
    tuple
        (second_shortest_length, count) - Length of second shortest path and count of paths with that length
    """
    # Use Dijkstra's algorithm to find distances to all nodes
    import heapq
    from collections import defaultdict
    
    # Track distances and path counts
    # For each node, we'll store (distance, count of paths with that distance)
    distances = {node: float('infinity') for node in G.nodes()}
    distances[source] = 0
    
    # For each node, we'll maintain a sorted list of (distance, count) pairs
    path_info = defaultdict(list)
    path_info[source].append((0, 1))  # (distance, count)
    
    # Priority queue for Dijkstra's algorithm
    pq = [(0, source)]
    
    while pq:
        dist, node = heapq.heappop(pq)
        
        # Skip outdated entries
        if dist > distances[node]:
            continue
            
        # Process neighbors
        for neighbor in G.neighbors(node):
            # Get edge weight (1 for unweighted graphs)
            weight = G[node][neighbor].get('weight', 1)
            
            # Calculate new distance
            new_dist = dist + weight
            
            # If we found a new distance to this neighbor
            if new_dist <= distances[neighbor]:
                # If it's a shorter path, update distance
                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    heapq.heappush(pq, (new_dist, neighbor))
                    path_info[neighbor] = [(new_dist, 0)]  # Reset path count
                
                # Count paths with this distance
                # Find paths to 'node' with distance = dist
                for d, count in path_info[node]:
                    if d == dist:
                        # Update count for the distance at neighbor
                        for i, (nd, nc) in enumerate(path_info[neighbor]):
                            if nd == new_dist:
                                path_info[neighbor][i] = (nd, nc + count)
                                break
                        else:
                            path_info[neighbor].append((new_dist, count))
    
    # Sort path_info for the target to get shortest and second shortest
    target_info = sorted(path_info[target])
    
    # If there's only one distance or no paths
    if len(target_info) <= 1:
        return None, 0
    
    # Return second shortest distance and its count
    return target_info[1][0], target_info[1][1]

In [47]:
def shortest_path_excluding_direct(G, source, target):
    """
    Find the shortest path between source and target,
    ignoring any direct edge between them.
    
    Parameters:
    -----------
    G : NetworkX graph
    source : node
        Starting node
    target : node
        Ending node
        
    Returns:
    --------
    list
        The shortest path excluding direct connection, or None if no such path exists
    """
    # Check if a direct edge exists
    has_direct_edge = G.has_edge(source, target)
    
    if has_direct_edge:
        # Temporarily remove the direct edge
        edge_data = None
        
        # For simple graphs
        edge_data = G.get_edge_data(source, target)
        G.remove_edge(source, target)
        
        try:
            # Find the shortest path without the direct edge
            path = list(nx.all_shortest_paths(G, source, target))
            return path
        except nx.NetworkXNoPath:
            # No alternate path exists
            return None
        finally:
            G.add_edge(source, target, **edge_data)

In [51]:
updates = []
c=0
for t in range(1950,2021):
    
    G=nx.read_gexf(nets_path+str(t-1)+"-"+str(t-9-1)+"_10Y.gexf")
    nodes=G.nodes
    # print(len(nodes))
    print("\r"+str(t))
    for i in df_y[df_y["tij"]==t].index:
        # c+=1
        # if c>50:
        #     break
        source=df_y.loc[i,"Author"]
        target=df_y.loc[i,"Coauthor"]
        source_conf=source in nodes
        target_conf=target in nodes
        if source_conf & target_conf:
            if nx.has_path(G, source, target):
                paths = list(nx.all_shortest_paths(G, source, target))
                distance=len(paths[0])-1  
                print(source)
                print(target) 
                print(paths)             
                pij=1/distance
                
                if distance==1:
                    excl=shortest_path_excluding_direct(G, source, target)
                    
                    if excl==None:
                        distance=float('inf')
                        count=0
                        pij=0
                    else:
                        distance=len(excl[0])-1
                        count=len(excl)
                        pij=1/distance
                        
                updates.append(df_y.iloc[i].to_dict()|
                                {   
                                    "tij":t,
                                    "tij-1":t-1,
                                    "Author":source,
                                    "Coauthor":target, 
                                    "distance": distance, 
                                    "pij_t":pij,
                                    "cij_t":len(paths),
                                    "A":source_conf, 
                                    "C":target_conf,
                                    "E":0
                                }
                            )
            else:
                updates.append(df_y.iloc[i].to_dict()|
                                {   
                                    "tij":t,
                                    "tij-1":t-1,
                                    "Author":source,
                                    "Coauthor":target, 
                                    "distance": float('inf'),
                                    "pij_t": 0,
                                    "cij_t":0,
                                    "A":source_conf, 
                                    "C":target_conf,
                                    "E":0
                                }
                            )
        else:
            if source_conf |target_conf:
                updates.append(df_y.iloc[i].to_dict()|
                                    {   
                                        "tij":t,
                                        "tij-1":t-1,
                                        "Author":source,
                                        "Coauthor":target, 
                                        "distance": float('inf'),
                                        "pij_t":0, 
                                        "cij_t":0,
                                        "A":source_conf, 
                                        "C":target_conf,
                                        "E":1
                                    }
                                )
            else:
                updates.append(df_y.iloc[i].to_dict()|
                                    {   
                                        "tij":t,
                                        "tij-1":t-1,
                                        "Author":source,
                                        "Coauthor":target, 
                                        "distance": float('inf'),
                                        "pij_t":0, 
                                        "cij_t":0,
                                        "A":source_conf, 
                                        "C":target_conf,
                                        "E":2
                                    })

1950
10060
10841
[['10060', '10841']]
10060
12993
[['10060', '12993']]
10616
12993
[['10616', '12993']]
10769
11949
[['10769', '11949']]
10780
14860
[['10780', '14860']]
10841
12993
[['10841', '12993']]
11192
14111
[['11192', '14111']]
11192
15406
[['11192', '15406']]
1123
10780
[['1123', '10780']]
1123
1288
[['1123', '1288']]
1123
14860
[['1123', '14860']]
1123
4150
[['1123', '4150']]
1123
4256
[['1123', '4256']]
1123
5319
[['1123', '5319']]
11800
11949
[['11800', '11949']]
11800
12434
[['11800', '12434']]
11800
12480
[['11800', '12480']]
11800
12993
[['11800', '12993']]
11800
13068
[['11800', '13068']]
11800
15045
[['11800', '15045']]
11800
15342
[['11800', '15342']]
11800
15790
[['11800', '15790']]
1184
13664
[['1184', '13664']]
1184
15045
[['1184', '15045']]
1184
15838
[['1184', '15838']]
1184
4249
[['1184', '4249']]
1184
5929
[['1184', '5929']]
11949
12434
[['11949', '12434']]
11949
12480
[['11949', '12480']]
11949
12993
[['11949', '12993']]
11949
13068
[['11949', '13068']]
11949


ZeroDivisionError: division by zero

In [ ]:
updates_5 = []
c=0
for t in range(1945,2021):
    G=nx.read_gexf(nets_path+str(t-1)+"-"+str(t-4-1)+"_10Y.gexf")
    nodes=G.nodes
    # print(len(nodes))
    print(t)
    for i in df_y[df_y["tij"]==t].index:
        # c+=1
        # if c>50:
        #     break
        source=df_y.loc[i,"Author"]
        target=df_y.loc[i,"Coauthor"]
        source_conf=source in nodes
        target_conf=target in nodes
        if source_conf & target_conf:
            if nx.has_path(G, source, target):
                paths = list(nx.all_shortest_paths(G, source, target))
                distance=len(paths[0])-1                
                pij=1/distance
                
                if distance==1:
                    excl=shortest_path_excluding_direct(G, source, target)
                    
                    if excl==None:
                        distance=float('inf')
                        count=0
                        pij=0
                    else:
                        distance=len(excl[0])-1
                        count=len(excl)
                        pij=1/distance
                        
                updates_5.append(df_y.iloc[i].to_dict()|
                                {   
                                    "tij":t,
                                    "tij-1":t-1,
                                    "Author":source,
                                    "Coauthor":target, 
                                    "distance": distance, 
                                    "pij_t":pij,
                                    "cij_t":len(paths),
                                    "A":source_conf, 
                                    "C":target_conf,
                                    "E":0
                                }
                            )
            else:
                updates_5.append(df_y.iloc[i].to_dict()|
                                {   
                                    "tij":t,
                                    "tij-1":t-1,
                                    "Author":source,
                                    "Coauthor":target, 
                                    "distance": float('inf'),
                                    "pij_t": 0,
                                    "cij_t":0,
                                    "A":source_conf, 
                                    "C":target_conf,
                                    "E":0
                                }
                            )
        else:
            if source_conf |target_conf:
                updates_5.append(df_y.iloc[i].to_dict()|
                                    {   
                                        "tij":t,
                                        "tij-1":t-1,
                                        "Author":source,
                                        "Coauthor":target, 
                                        "distance": float('inf'),
                                        "pij_t":0, 
                                        "cij_t":0,
                                        "A":source_conf, 
                                        "C":target_conf,
                                        "E":1
                                    }
                                )
            else:
                updates_5.append(df_y.iloc[i].to_dict()|
                                    {   
                                        "tij":t,
                                        "tij-1":t-1,
                                        "Author":source,
                                        "Coauthor":target, 
                                        "distance": float('inf'),
                                        "pij_t":0, 
                                        "cij_t":0,
                                        "A":source_conf, 
                                        "C":target_conf,
                                        "E":2
                                    })

1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [ ]:
updates_20 = []
c=0
for t in range(1960,2021):
    G=nx.read_gexf(nets_path+str(t-1)+"-"+str(t-19-1)+"_10Y.gexf")
    nodes=G.nodes
    # print(len(nodes))
    print(t)
    for i in df_y[df_y["tij"]==t].index:
        # c+=1
        # if c>50:
        #     break
        source=df_y.loc[i,"Author"]
        target=df_y.loc[i,"Coauthor"]
        source_conf=source in nodes
        target_conf=target in nodes
        if source_conf & target_conf:
            if nx.has_path(G, source, target):
                paths = list(nx.all_shortest_paths(G, source, target))
                distance=len(paths[0])-1                
                pij=1/distance
                
                if distance==1:
                    excl=shortest_path_excluding_direct(G, source, target)
                    
                    if excl==None:
                        distance=float('inf')
                        count=0
                        pij=0
                    else:
                        distance=len(excl[0])-1
                        count=len(excl)
                        pij=1/distance
                        
                updates_20.append(df_y.iloc[i].to_dict()|
                                {   
                                    "tij":t,
                                    "tij-1":t-1,
                                    "Author":source,
                                    "Coauthor":target, 
                                    "distance": distance, 
                                    "pij_t":pij,
                                    "cij_t":len(paths),
                                    "A":source_conf, 
                                    "C":target_conf,
                                    "E":0
                                }
                            )
            else:
                updates_20.append(df_y.iloc[i].to_dict()|
                                {   
                                    "tij":t,
                                    "tij-1":t-1,
                                    "Author":source,
                                    "Coauthor":target, 
                                    "distance": float('inf'),
                                    "pij_t": 0,
                                    "cij_t":0,
                                    "A":source_conf, 
                                    "C":target_conf,
                                    "E":0
                                }
                            )
        else:
            if source_conf |target_conf:
                updates_20.append(df_y.iloc[i].to_dict()|
                                    {   
                                        "tij":t,
                                        "tij-1":t-1,
                                        "Author":source,
                                        "Coauthor":target, 
                                        "distance": float('inf'),
                                        "pij_t":0, 
                                        "cij_t":0,
                                        "A":source_conf, 
                                        "C":target_conf,
                                        "E":1
                                    }
                                )
            else:
                updates_20.append(df_y.iloc[i].to_dict()|
                                    {   
                                        "tij":t,
                                        "tij-1":t-1,
                                        "Author":source,
                                        "Coauthor":target, 
                                        "distance": float('inf'),
                                        "pij_t":0, 
                                        "cij_t":0,
                                        "A":source_conf, 
                                        "C":target_conf,
                                        "E":2
                                    })

1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [ ]:
upd=pd.DataFrame(updates)
upd_5=pd.DataFrame(updates_5)
upd_20=pd.DataFrame(updates_20)

# upd.to_pickle("upd.pkl")


In [ ]:
upd.head()

,Author,Coauthor,tij,ytij,tij_1,tij_2,tij_0,tij-1,distance,pij_t,cij_t,A,C,E
0,10023,15700,1950,0,1952,1952,1940,1949,inf,0.0,0,True,True,0
1,10177,12194,1950,0,1956,1972,1950,1949,inf,0.0,0,True,False,1
2,10333,10980,1950,0,1946,1957,1940,1949,2.0,0.5,1,True,True,0
3,10333,11857,1950,0,1946,1957,1946,1949,2.0,0.5,1,True,True,0
4,10333,12583,1950,0,1946,1952,1941,1949,2.0,0.5,1,True,True,0


In [ ]:
a1_lags_points.head()

,a1_order_str,points20,year,c_counts20,p_counts20,points10,c_counts10,p_counts10,points5,c_counts5,p_counts5
0,10023,4.500000,1940,0,2,4.500000,0.0,2.0,4.500000,0.0,2.0
1,10055,5.500000,1940,0,1,5.500000,0.0,1.0,5.500000,0.0,1.0
2,10125,5.500000,1940,0,1,5.500000,0.0,1.0,5.500000,0.0,1.0
3,10263,11.333333,1940,1,2,11.333333,1.0,2.0,11.333333,1.0,2.0
4,10273,3.000000,1940,0,1,3.000000,0.0,1.0,3.000000,0.0,1.0


In [ ]:
upd_w_lags_10=upd.merge(a1_lags_points[["a1_order_str","year", "points10","c_counts10","p_counts10"]], left_on=["Author", "tij-1"], right_on=["a1_order_str", "year"], how="left").rename(columns={"points10":"p-a", "c_counts10":"co-a","p_counts10":"pa-a"}).merge(a1_lags_points[["a1_order_str","year", "points10","c_counts10","p_counts10"]], left_on=["Coauthor", "tij-1"], right_on=["a1_order_str", "year"], how="left").rename(columns={"points10":"p-c","c_counts10":"co-c","p_counts10":"pa-c"}).drop(columns=["a1_order_str_x", "year_x","a1_order_str_y", "year_y"]).fillna(0)
upd_w_lags_5=upd.merge(a1_lags_points[["a1_order_str","year", "points5","c_counts5","p_counts5"]], left_on=["Author", "tij-1"], right_on=["a1_order_str", "year"], how="left").rename(columns={"points5":"p-a","c_counts5":"co-a","p_counts5":"pa-a"}).merge(a1_lags_points[["a1_order_str","year", "points5", "c_counts5","p_counts5"]], left_on=["Coauthor", "tij-1"], right_on=["a1_order_str", "year"], how="left").rename(columns={"points5":"p-c", "c_counts5":"co-c","p_counts5":"pa-c"}).drop(columns=["a1_order_str_x", "year_x","a1_order_str_y", "year_y"]).fillna(0)
upd_w_lags_20=upd.merge(a1_lags_points[["a1_order_str","year", "points20","c_counts20","p_counts20"]], left_on=["Author", "tij-1"], right_on=["a1_order_str", "year"], how="left").rename(columns={"points20":"p-a", "c_counts20":"co-a","p_counts20":"pa-a"}).merge(a1_lags_points[["a1_order_str","year", "points20", "c_counts20","p_counts20"]], left_on=["Coauthor", "tij-1"], right_on=["a1_order_str", "year"], how="left").rename(columns={"points20":"p-c", "c_counts20":"co-c","p_counts20":"pa-c"}).drop(columns=["a1_order_str_x", "year_x","a1_order_str_y", "year_y"]).fillna(0)

In [ ]:
upd_w_lags_10.head()

,Author,Coauthor,tij,ytij,tij_1,tij_2,tij_0,tij-1,distance,pij_t,cij_t,A,C,E,p-a,co-a,pa-a,p-c,co-c,pa-c
0,10023,15700,1950,0,1952,1952,1940,1949,inf,0.0,0,True,True,0,7.000000,0.0,3.0,14.833333,2.0,3.0
1,10177,12194,1950,0,1956,1972,1950,1949,inf,0.0,0,True,False,1,7.833333,1.0,3.0,0.000000,0.0,0.0
2,10333,10980,1950,0,1946,1957,1940,1949,2.0,0.5,1,True,True,0,17.750000,18.0,4.0,104.750000,18.0,16.0
3,10333,11857,1950,0,1946,1957,1946,1949,2.0,0.5,1,True,True,0,17.750000,18.0,4.0,0.250000,18.0,1.0
4,10333,12583,1950,0,1946,1952,1941,1949,2.0,0.5,1,True,True,0,17.750000,18.0,4.0,16.812500,32.0,6.0


In [ ]:
upd_w_lags_10["avg_p"]=(upd_w_lags_10["p-a"]+upd_w_lags_10["p-c"])/2
upd_w_lags_10["abs_p"]=abs(upd_w_lags_10["p-a"]-upd_w_lags_10["p-c"])
upd_w_lags_10["c-a"]=upd_w_lags_10["co-a"]/upd_w_lags_10["pa-a"]
upd_w_lags_10["c-c"]=upd_w_lags_10["co-c"]/upd_w_lags_10["pa-c"]
upd_w_lags_10=upd_w_lags_10.fillna(0)
upd_w_lags_10["avg_c"]=(upd_w_lags_10["c-a"]+upd_w_lags_10["c-c"])/2
upd_w_lags_10["abs_c"]=abs(upd_w_lags_10["c-a"]-upd_w_lags_10["c-c"])


upd_w_lags_20["avg_p"]=(upd_w_lags_20["p-a"]+upd_w_lags_20["p-c"])/2
upd_w_lags_20["abs_p"]=abs(upd_w_lags_20["p-a"]-upd_w_lags_20["p-c"])
upd_w_lags_20["c-a"]=upd_w_lags_20["co-a"]/upd_w_lags_20["pa-a"]
upd_w_lags_20["c-c"]=upd_w_lags_20["co-c"]/upd_w_lags_20["pa-c"]
upd_w_lags_20=upd_w_lags_20.fillna(0)
upd_w_lags_20["avg_c"]=(upd_w_lags_20["c-a"]+upd_w_lags_20["c-c"])/2
upd_w_lags_20["abs_c"]=abs(upd_w_lags_20["c-a"]-upd_w_lags_20["c-c"])

upd_w_lags_5["avg_p"]=(upd_w_lags_5["p-a"]+upd_w_lags_5["p-c"])/2
upd_w_lags_5["abs_p"]=abs(upd_w_lags_5["p-a"]-upd_w_lags_5["p-c"])
upd_w_lags_5["c-a"]=upd_w_lags_5["co-a"]/upd_w_lags_5["pa-a"]
upd_w_lags_5["c-c"]=upd_w_lags_5["co-c"]/upd_w_lags_5["pa-c"]
upd_w_lags_5=upd_w_lags_5.fillna(0)
upd_w_lags_5["avg_c"]=(upd_w_lags_5["c-a"]+upd_w_lags_5["c-c"])/2
upd_w_lags_5["abs_c"]=abs(upd_w_lags_5["c-a"]-upd_w_lags_5["c-c"])


In [ ]:
upd_w_lags_10.to_pickle("flattened_co-author_10.pkl")
upd_w_lags_20.to_pickle("flattened_co-author_20.pkl")
upd_w_lags_5.to_pickle("flattened_co-author_5.pkl")


In [ ]:

upd_w_lags_10.dropna().to_csv("flattened_co-author_10.csv")
upd_w_lags_20.dropna().to_csv("flattened_co-author_20.csv")
upd_w_lags_5.dropna().to_csv("flattened_co-author_5.csv")

In [ ]:
upd_w_lags_10.head()

,Author,Coauthor,tij,ytij,tij_1,tij_2,tij_0,tij-1,distance,pij_t,...,pa-a,p-c,co-c,pa-c,avg_p,abs_p,c-a,c-c,avg_c,abs_c
0,10023,15700,1950,0,1952,1952,1940,1949,inf,0.0,...,3.0,14.833333,2.0,3.0,10.916667,7.833333,0.000000,0.666667,0.333333,0.666667
1,10177,12194,1950,0,1956,1972,1950,1949,inf,0.0,...,3.0,0.000000,0.0,0.0,3.916667,7.833333,0.333333,0.000000,0.166667,0.333333
2,10333,10980,1950,0,1946,1957,1940,1949,2.0,0.5,...,4.0,104.750000,18.0,16.0,61.250000,87.000000,4.500000,1.125000,2.812500,3.375000
3,10333,11857,1950,0,1946,1957,1946,1949,2.0,0.5,...,4.0,0.250000,18.0,1.0,9.000000,17.500000,4.500000,18.000000,11.250000,13.500000
4,10333,12583,1950,0,1946,1952,1941,1949,2.0,0.5,...,4.0,16.812500,32.0,6.0,17.281250,0.937500,4.500000,5.333333,4.916667,0.833333


In [ ]:
upd_w_lags_10.shape

(106190, 26)

In [ ]:
upd_w_lags_10[upd_w_lags_10["pa-c"]==0].head()

,Author,Coauthor,tij,ytij,tij_1,tij_2,tij_0,tij-1,distance,pij_t,...,pa-a,p-c,co-c,pa-c,avg_p,abs_p,c-a,c-c,avg_c,abs_c
1,10177,12194,1950,0,1956,1972,1950,1949,inf,0.0,...,3.0,0.0,0.0,0.0,3.916667,7.833333,0.333333,0.0,0.166667,0.333333
32,12553,14396,1950,1,1950,1950,1950,1949,inf,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
39,1273,15278,1950,1,1950,1950,1950,1949,inf,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
49,13861,15423,1950,1,1950,1950,1950,1949,inf,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
180,5881,8596,1950,1,1950,1975,1950,1949,inf,0.0,...,1.0,0.0,0.0,0.0,5.000000,10.000000,0.000000,0.0,0.000000,0.000000


In [ ]:
# upd=pd.read_pickle("upd.pkl")

In [ ]:
df_y_a=upd_w_lags_10
df_y_b=df_y_a.reset_index(drop=True).fillna(0).drop_duplicates(keep="first").reset_index(drop=True)
df_f=df_y_b[df_y_b["tij"]<=df_y_b["tij_1"]].reset_index(drop=True)
df_c=df_y_b[df_y_b["tij"]>df_y_b["tij_1"]].reset_index(drop=True)

In [ ]:
df_y_b_5=upd_w_lags_5.reset_index(drop=True).fillna(0).drop_duplicates(keep="first").reset_index(drop=True)
df_f_5=df_y_b_5[df_y_b_5["tij"]<=df_y_b_5["tij_1"]].reset_index(drop=True)
df_c_5=df_y_b_5[df_y_b_5["tij"]>df_y_b_5["tij_1"]].reset_index(drop=True)

In [ ]:
df_y_b_20=upd_w_lags_20.reset_index(drop=True).fillna(0).drop_duplicates(keep="first").reset_index(drop=True)
df_f_20=df_y_b_20[df_y_b_20["tij"]<=df_y_b_20["tij_1"]].reset_index(drop=True)
df_c_20=df_y_b_20[df_y_b_20["tij"]>df_y_b_20["tij_1"]].reset_index(drop=True)

In [ ]:
# Select a numerical column for histogram (replace 'column_name' with actual column)
def plothist(df,column_name):
    # Plot histogram
    plt.figure(figsize=(8, 5))
    plt.hist(df[column_name], bins=30, color='blue', alpha=0.7, edgecolor='black')
    plt.xlabel(column_name)
    plt.ylabel("Frequency")
    plt.title(f"Histogram of {column_name}")
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Show plot
    plt.show()


def conf_stat(conf):
    tn, fp, fn, tp = conf.ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(str(tn)+"true neg")
    print(str(fp)+"false pos")
    print(str(fn)+"false neg")
    print(str(tp)+"true pos")
    # Multi-class:
    precisions = np.zeros(conf.shape[0])
    recalls = np.zeros(conf.shape[0])
    
    for i in range(conf.shape[0]):
        # Calculate precision and recall for class i
        true_pos = conf[i, i]
        false_pos = sum(conf[:, i]) - true_pos
        false_neg = sum(conf[i, :]) - true_pos
        
        # Handle division by zero
        precisions[i] = true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0
        recalls[i] = true_pos / (true_pos + false_neg) if (true_pos + false_neg) > 0 else 0

    # Calculate averages if needed
    macro_precision = np.mean(precisions)
    macro_recall = np.mean(recalls)


# Function to calculate statistics for logistic regression
def get_logistic_regression_stats(model, X, feature_names=None):
    """
    Calculate p-values and significance levels for logistic regression coefficients.
    
    Parameters:
    -----------
    model : fitted sklearn.linear_model.LogisticRegression
        The fitted logistic regression model
    X : array-like
        The input features used to train the model
    feature_names : list, optional
        Names of the features (columns)
        
    Returns:
    --------
    DataFrame with coefficients, standard errors, z-values, p-values, and significance indicators
    """
    # Get coefficients and intercept
    coef = model.coef_[0]
    intercept = model.intercept_[0]
    
    # Combine coefficients with intercept
    params = np.append(intercept, coef)
    
    # Calculate predictions and probabilities
    y_pred = model.predict(X)
    probs = model.predict_proba(X)
    
    # Design matrix with intercept
    X_design = np.hstack([np.ones((X.shape[0], 1)), X])
    
    # Calculate variance-covariance matrix
    # For logistic regression, V is a diagonal matrix of p_i * (1 - p_i)
    p = probs[:, 1]
    V = np.diagflat(p * (1 - p))
    
    # Calculate standard errors
    covb = np.linalg.inv(X_design.T @ V @ X_design)
    se = np.sqrt(np.diag(covb))
    
    # Calculate z-scores (for logistic regression, we use z instead of t)
    z_scores = params / se
    
    # Calculate p-values (two-tailed test)
    p_values = 2 * (1 - stats.norm.cdf(abs(z_scores)))

    degrees_of_freedom = X.shape[0] - X.shape[1]
    t_scores = params / se
    p_values_t = 2 * (1 - stats.t.cdf(abs(t_scores), df=degrees_of_freedom))
    
    # Create significance indicators
    significance = [''] * len(p_values)
    for i, p in enumerate(p_values):
        if p < 0.001:
            significance[i] = '***'
        elif p < 0.01:
            significance[i] = '**'
        elif p < 0.05:
            significance[i] = '*'
        elif p < 0.1:
            significance[i] = '.'

    significance_t = [''] * len(p_values_t)
    for i, p in enumerate(p_values_t):
        if p < 0.001:
            significance_t[i] = '***'
        elif p < 0.01:
            significance_t[i] = '**'
        elif p < 0.05:
            significance_t[i] = '*'
        elif p < 0.1:
            significance_t[i] = '.'
    
    # Create feature names if not provided
    if feature_names is None:
        feature_names = [f'X{i}' for i in range(X.shape[1])]
    
    # Combine results into a DataFrame
    result = pd.DataFrame({
        'Variable': ['Intercept'] + list(feature_names),
        'Coefficient': params,
        'Std. Error': se,
        'z-value': z_scores,
        'p-value': p_values,
        'Significance': significance,
        't-value':t_scores,
        'p_values_t':p_values_t,
        "Significance_t":significance_t
    })
    
    return result


In [ ]:
def logi_mod(df, y, e, st, en):
    
    # Original logistic regression code
    cat_var = df[['E']]
    
    # Create and apply encoder
    encoder = OneHotEncoder(drop='first', sparse_output=False)  # Drop first category to avoid multicollinearity
    encoded = encoder.fit_transform(cat_var)
    
    # Create DataFrame with encoded variables
    encoded_df = pd.DataFrame(
        encoded,
        columns=encoder.get_feature_names_out(['E'])
    )
    
    # Combine with other features
    if e:
        X = pd.concat([df.drop('E', axis=1), encoded_df], axis=1)
    else:
        X=df.drop('E', axis=1)
    
    model = LogisticRegression(fit_intercept=True)
    model.fit(X, y)
    print(X.shape)

    # Count of positive outcomes
    print(f"\nPercentage of positive outcomes: {y.mean() * 100:.2f}%")
    # std_scaler=StandardScaler().fit(X)
    # X_std=std_scaler.transform(X)

    model=LogisticRegression(fit_intercept=True, max_iter=30000, solver="newton-cg")
    # Fit logistic regression model
    # model = LogisticRegression(
                            # penalty=None, 
                            # solver='newton-cg', 
                            # fit_intercept=True)  # No regularization - will result in similar outcome to statsmodels
    model.fit(X, y)

    # Calculate and display statistics
    stats_df = get_logistic_regression_stats(model, X, feature_names=X.columns)

    # Print the results
    print("\nLogistic Regression Results:")
    print(stats_df)

    # Print the true coefficients for comparison
    # print("\nTrue Coefficients:")
    # print(pd.Series({'Intercept': true_intercept, **true_coefficients}))
    
    y_pred = model.predict(X)
    print("Classification Report:")
    print(classification_report(y, y_pred, zero_division=1))
    
    print("Confusion Matrix:")
    conf=confusion_matrix(y, y_pred)
    print(conf)

    predictions = model.predict(X)

    # newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X.reset_index(drop=True)))
    MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))
    print(MSE)
    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    
    return (classification_report(y, y_pred, zero_division=1),conf, stats_df)

def print_latex_format(mod):
    reshaped={}
    for i in range(len(mod)):
        res=mod[i][2]
        # print(res)
        # print(type(res))
        # print(res.index)
        dec=6
        just="c"
        for j in res.index:
            if res.loc[j,"Variable"] not in reshaped.keys():
                reshaped[res.loc[j,"Variable"]]="&\\makecell["+just+"]{"+str(round(res.loc[j, "Coefficient"],dec)) + str(res.loc[j, "Significance"]) +'\\\\(' +str(round(res.loc[j,'Std. Error'],dec)) +')\\\\' + str(round(res.loc[j, "p-value"],dec))+"} "
            else:
                reshaped[res.loc[j,"Variable"]]+="& \\makecell["+just+"]{"+str(round(res.loc[j, "Coefficient"],dec)) + str(res.loc[j, "Significance"]) +'\\\\(' +str(round(res.loc[j,'Std. Error'],dec)) +')\\\\' + str(round(res.loc[j, "p-value"],dec))+"}"

    for i in reshaped.keys():
        print("\makecell[l]{"+i.replace("_"," ")+"}"+reshaped[i]+"\\\\")

In [ ]:
df_f.columns

Index(['Author', 'Coauthor', 'tij', 'ytij', 'tij_1', 'tij_2', 'tij_0', 'tij-1',
       'distance', 'pij_t', 'cij_t', 'A', 'C', 'E', 'p-a', 'co-a', 'pa-a',
       'p-c', 'co-c', 'pa-c', 'avg_p', 'abs_p', 'c-a', 'c-c', 'avg_c',
       'abs_c'],
      dtype='object')

In [ ]:
mods_d=[]
for i in range(1960,2021,10):
    print("#########################################")
    print(str(i-10)+' to '+str(i-1) + " inclusive")
    print("#########################################")
    # test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)].reset_index(drop=True)
    test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)][["pij_t",'cij_t',"ytij","tij","tij_0","tij_1","Author", "Coauthor","E","avg_p","abs_p",'avg_c',"abs_c"]].drop_duplicates().reset_index(drop=True)
    # plothist(test_df, "pij_t")
    test_df["log(cij)"]=np.log(test_df['cij_t'].clip(lower=0.0001))
    # plothist(test_df, "E")
    X=test_df[["pij_t",'log(cij)','E']].fillna(0)
    y=test_df["ytij"]
    print()
    mods_d.append(logi_mod(X,y, False, i-10,i-1 ))

#########################################
1950 to 1959 inclusive
#########################################

(767, 2)

Percentage of positive outcomes: 38.46%

Logistic Regression Results:
    Variable  Coefficient  Std. Error   z-value   p-value Significance  \
0  Intercept    -1.166831    3.098520 -0.376577  0.706488                
1      pij_t     0.200927    7.530531  0.026682  0.978714                
2   log(cij)    -0.076303    0.336515 -0.226743  0.820623                

    t-value  p_values_t Significance_t  
0 -0.376577    0.706592                 
1  0.026682    0.978721                 
2 -0.226743    0.820684                 
Classification Report:
              precision    recall  f1-score   support

           0       0.62      1.00      0.76       472
           1       1.00      0.00      0.00       295

    accuracy                           0.62       767
   macro avg       0.81      0.50      0.38       767
weighted avg       0.76      0.62      0.47       767

C

In [749]:
print_latex_format(mods_d)


\makecell[l]{Intercept}&\makecell[c]{-1.166831\\(3.09852)\\0.706488} & \makecell[c]{-0.635018\\(4.294779)\\0.882455}& \makecell[c]{-1.387951.\\(0.748825)\\0.06381}& \makecell[c]{-2.057708***\\(0.268372)\\0.0}& \makecell[c]{-2.010615***\\(0.127534)\\0.0}& \makecell[c]{-2.110601***\\(0.091824)\\0.0}& \makecell[c]{-1.308137***\\(0.071065)\\0.0}\\
\makecell[l]{pij t}&\makecell[c]{0.200927\\(7.530531)\\0.978714} & \makecell[c]{0.072408\\(9.179712)\\0.993707}& \makecell[c]{-0.271819\\(2.230505)\\0.903006}& \makecell[c]{1.078691\\(0.797777)\\0.176336}& \makecell[c]{0.248283\\(0.530295)\\0.639643}& \makecell[c]{0.599184\\(0.39687)\\0.131101}& \makecell[c]{0.260846\\(0.278091)\\0.348249}\\
\makecell[l]{log(cij)}&\makecell[c]{-0.076303\\(0.336515)\\0.820623} & \makecell[c]{-0.022327\\(0.466337)\\0.961814}& \makecell[c]{-0.081081\\(0.081381)\\0.319096}& \makecell[c]{-0.125624***\\(0.029269)\\1.8e-05}& \makecell[c]{-0.128059***\\(0.014084)\\0.0}& \makecell[c]{-0.149606***\\(0.010195)\\0.0}& \makec

In [752]:
mods_e=[]
for i in range(1960,2021,10):
    print("#########################################")
    print(str(i-10)+' to '+str(i-1) + " inclusive")
    print("#########################################")
    # test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)].reset_index(drop=True)
    test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)][["pij_t",'cij_t',"ytij","tij","tij_0","tij_1","Author", "Coauthor","E","avg_p","abs_p",'avg_c',"abs_c"]].drop_duplicates().reset_index(drop=True)
    # plothist(test_df, "pij_t")
    # plothist(test_df, "E")
    test_df["log(cij)"]=np.log(test_df['cij_t'].clip(lower=0.0001))

    X=test_df[["pij_t",'log(cij)',"E","abs_p", "avg_p",'abs_c',"avg_c"]].fillna(0)
    y=test_df["ytij"]
    print()
    mods_e.append(logi_mod(X,y, False, i-10,i-1 ))

#########################################
1950 to 1959 inclusive
#########################################

(767, 6)

Percentage of positive outcomes: 38.46%

Logistic Regression Results:
    Variable  Coefficient  Std. Error   z-value       p-value Significance  \
0  Intercept     1.037568    3.111562  0.333456  7.387904e-01                
1      pij_t     0.268692    7.893370  0.034040  9.728451e-01                
2   log(cij)     0.095716    0.337657  0.283470  7.768165e-01                
3      abs_p     0.032673    0.007007  4.663164  3.113843e-06          ***   
4      avg_p    -0.077822    0.012107 -6.428045  1.292559e-10          ***   
5      abs_c     0.760855    0.329151  2.311565  2.080166e-02            *   
6      avg_c    -1.559469    0.654230 -2.383671  1.714093e-02            *   

    t-value    p_values_t Significance_t  
0  0.333456  7.388823e-01                 
1  0.034040  9.728540e-01                 
2  0.283470  7.768936e-01                 
3  4.663164  3.

In [753]:
print_latex_format(mods_e)

\makecell[l]{Intercept}&\makecell[c]{1.037568\\(3.111562)\\0.73879} & \makecell[c]{1.274863\\(4.642626)\\0.783624}& \makecell[c]{1.988289*\\(0.9082)\\0.028578}& \makecell[c]{0.53497.\\(0.286176)\\0.061571}& \makecell[c]{0.690116***\\(0.150591)\\5e-06}& \makecell[c]{0.273515*\\(0.111121)\\0.013839}& \makecell[c]{0.672808***\\(0.087207)\\0.0}\\
\makecell[l]{pij t}&\makecell[c]{0.268692\\(7.89337)\\0.972845} & \makecell[c]{0.071574\\(9.824579)\\0.994187}& \makecell[c]{-0.80681\\(2.437809)\\0.740677}& \makecell[c]{1.098214\\(0.804717)\\0.172341}& \makecell[c]{1.62742**\\(0.547011)\\0.002929}& \makecell[c]{1.698268***\\(0.397617)\\1.9e-05}& \makecell[c]{1.962396***\\(0.282341)\\0.0}\\
\makecell[l]{log(cij)}&\makecell[c]{0.095716\\(0.337657)\\0.776816} & \makecell[c]{0.075171\\(0.503936)\\0.881421}& \makecell[c]{0.17718.\\(0.098187)\\0.07115}& \makecell[c]{0.042209\\(0.030334)\\0.164084}& \makecell[c]{0.035902*\\(0.015298)\\0.018931}& \makecell[c]{-0.001303\\(0.011172)\\0.907187}& \makecell[

In [754]:
mods_e=[]
for i in range(1960,2021,10):
    print("#########################################")
    print(str(i-10)+' to '+str(i-1) + " inclusive")
    print("#########################################")
    # test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)].reset_index(drop=True)
    test_df=df_f_5[(df_f_5["tij"]<i)&(df_f_5["tij"]>=i-10)][["pij_t",'cij_t',"ytij","tij","tij_0","tij_1","Author", "Coauthor","E","avg_p","abs_p",'avg_c',"abs_c"]].drop_duplicates().reset_index(drop=True)
    # plothist(test_df, "pij_t")
    # plothist(test_df, "E")
    test_df["log(cij)"]=np.log(test_df['cij_t'].clip(lower=0.0001))

    X=test_df[["pij_t",'log(cij)',"E","abs_p", "avg_p",'abs_c',"avg_c"]].fillna(0)
    y=test_df["ytij"]
    print()
    mods_e.append(logi_mod(X,y, False, i-10,i-1 ))

#########################################
1950 to 1959 inclusive
#########################################

(767, 6)

Percentage of positive outcomes: 38.46%

Logistic Regression Results:
    Variable  Coefficient  Std. Error   z-value       p-value Significance  \
0  Intercept     0.874665    3.207398  0.272703  7.850819e-01                
1      pij_t     0.322294    8.820408  0.036540  9.708521e-01                
2   log(cij)     0.081085    0.348033  0.232980  8.157766e-01                
3      abs_p     0.062692    0.011865  5.283903  1.264600e-07          ***   
4      avg_p    -0.143153    0.021033 -6.806087  1.002887e-11          ***   
5      abs_c     0.727974    0.456120  1.596015  1.104854e-01                
6      avg_c    -1.519775    0.907404 -1.674859  9.396184e-02            .   

    t-value    p_values_t Significance_t  
0  0.272703  7.851559e-01                 
1  0.036540  9.708617e-01                 
2  0.232980  8.158393e-01                 
3  5.283903  1.

In [755]:
print_latex_format(mods_e)

\makecell[l]{Intercept}&\makecell[c]{0.874665\\(3.207398)\\0.785082} & \makecell[c]{0.830591\\(5.612942)\\0.88236}& \makecell[c]{1.292185\\(0.846786)\\0.127013}& \makecell[c]{-0.070313\\(0.285362)\\0.805373}& \makecell[c]{-0.327112*\\(0.140772)\\0.020141}& \makecell[c]{-0.569374***\\(0.10269)\\0.0}& \makecell[c]{-0.119518\\(0.08003)\\0.13533}\\
\makecell[l]{pij t}&\makecell[c]{0.322294\\(8.820408)\\0.970852} & \makecell[c]{0.042052\\(11.667713)\\0.997124}& \makecell[c]{-0.656479\\(2.321445)\\0.777339}& \makecell[c]{1.073645\\(0.823726)\\0.192438}& \makecell[c]{1.483667**\\(0.552446)\\0.007239}& \makecell[c]{1.185699**\\(0.400117)\\0.003043}& \makecell[c]{1.545623***\\(0.28589)\\0.0}\\
\makecell[l]{log(cij)}&\makecell[c]{0.081085\\(0.348033)\\0.815777} & \makecell[c]{0.043531\\(0.609276)\\0.943042}& \makecell[c]{0.106464\\(0.091534)\\0.244783}& \makecell[c]{-0.002679\\(0.030445)\\0.92989}& \makecell[c]{-0.03564*\\(0.0147)\\0.015332}& \makecell[c]{-0.062639***\\(0.010617)\\0.0}& \makecel

In [756]:
mods_e=[]
for i in range(1970,2021,10):
    print("#########################################")
    print(str(i-10)+' to '+str(i-1) + " inclusive")
    print("#########################################")
    # test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)].reset_index(drop=True)
    test_df=df_f_20[(df_f_20["tij"]<i)&(df_f_20["tij"]>=i-10)][["pij_t",'cij_t',"ytij","tij","tij_0","tij_1","Author", "Coauthor","E","avg_p","abs_p",'avg_c',"abs_c"]].drop_duplicates().reset_index(drop=True)
    # plothist(test_df, "pij_t")
    # plothist(test_df, "E")
    test_df["log(cij)"]=np.log(test_df['cij_t'].clip(lower=0.0001))

    X=test_df[["pij_t",'log(cij)',"E","abs_p", "avg_p",'abs_c',"avg_c"]].fillna(0)
    y=test_df["ytij"]
    print()
    mods_e.append(logi_mod(X,y, False, i-10,i-1 ))

#########################################
1960 to 1969 inclusive
#########################################

(1439, 6)

Percentage of positive outcomes: 39.40%

Logistic Regression Results:
    Variable  Coefficient  Std. Error    z-value       p-value Significance  \
0  Intercept     1.068705    4.625020   0.231070  8.172602e-01                
1      pij_t     0.154426    9.796045   0.015764  9.874226e-01                
2   log(cij)     0.054405    0.502051   0.108365  9.137061e-01                
3      abs_p     0.047764    0.006318   7.560490  4.019007e-14          ***   
4      avg_p    -0.120018    0.011704 -10.254365  0.000000e+00          ***   
5      abs_c     0.204271    0.277721   0.735525  4.620195e-01                
6      avg_c    -0.794049    0.491304  -1.616206  1.060497e-01                

     t-value    p_values_t Significance_t  
0   0.231070  8.172932e-01                 
1   0.015764  9.874248e-01                 
2   0.108365  9.137213e-01                 
3 

In [757]:
print_latex_format(mods_e)

\makecell[l]{Intercept}&\makecell[c]{1.068705\\(4.62502)\\0.81726} & \makecell[c]{2.213505*\\(1.015984)\\0.029355}& \makecell[c]{0.694524*\\(0.287533)\\0.015716}& \makecell[c]{0.772414***\\(0.15013)\\0.0}& \makecell[c]{0.482107***\\(0.111276)\\1.5e-05}& \makecell[c]{0.822898***\\(0.08823)\\0.0}\\
\makecell[l]{pij t}&\makecell[c]{0.154426\\(9.796045)\\0.987423} & \makecell[c]{-1.008954\\(2.574108)\\0.695086}& \makecell[c]{0.858051\\(0.799828)\\0.283363}& \makecell[c]{1.48735**\\(0.545252)\\0.006375}& \makecell[c]{1.739281***\\(0.400275)\\1.4e-05}& \makecell[c]{1.792317***\\(0.282494)\\0.0}\\
\makecell[l]{log(cij)}&\makecell[c]{0.054405\\(0.502051)\\0.913706} & \makecell[c]{0.201767.\\(0.110007)\\0.066634}& \makecell[c]{0.051423.\\(0.030474)\\0.091517}& \makecell[c]{0.029729*\\(0.015068)\\0.048491}& \makecell[c]{-0.001801\\(0.010999)\\0.86992}& \makecell[c]{-0.030453***\\(0.008571)\\0.000381}\\
\makecell[l]{abs p}&\makecell[c]{0.047764***\\(0.006318)\\0.0} & \makecell[c]{0.047529***\\(0.

In [767]:
test_df["cij_t"].unique()

array([0, 1])

In [766]:
test_df[test_df["pij_t"]>0].head()

,pij_t,cij_t,ytij,tij,tij_0,tij_1,Author,Coauthor,E,avg_p,abs_p,avg_c,abs_c,log(cij),pij:log(cij)
62,0.333333,1,0,1950,1946,1952,5386,13275,0,18.541667,3.583333,2.321429,4.357143,0.0,0.0
89,0.500000,1,0,1951,1946,1952,13164,13275,0,52.125000,62.750000,1.966667,3.266667,0.0,0.0
90,0.500000,1,0,1951,1946,1952,13275,15421,0,54.041667,66.583333,2.027273,3.145455,0.0,0.0
141,0.250000,1,0,1951,1940,1952,5386,13164,0,51.166667,64.666667,0.250000,0.166667,0.0,0.0
142,0.333333,1,0,1951,1946,1952,5386,13275,0,19.791667,1.916667,1.883333,3.433333,0.0,0.0


In [769]:
mods_e=[]
for i in range(1960,2021,10):
    print("#########################################")
    print(str(i-10)+' to '+str(i-1) + " inclusive")
    print("#########################################")
    # test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)].reset_index(drop=True)
    test_df=df_f[(df_f["tij"]<i)&(df_f["tij"]>=i-10)][["pij_t",'cij_t',"ytij","tij","tij_0","tij_1","Author", "Coauthor","E","avg_p","abs_p",'avg_c',"abs_c"]].drop_duplicates().reset_index(drop=True)
    # plothist(test_df, "pij_t")
    # plothist(test_df, "E")
    print(test_df["cij_t"].unique())
    test_df["log(cij)"]=np.log(test_df['cij_t'].clip(lower=0.0001))
    test_df=test_df.fillna(0)
    # test_df["pij:log(cij)"]=test_df["pij_t"]*test_df["log(cij)"]
    # test_df.loc[test_df["pij:log(cij)"]==-0,"pij:log(cij)
    # "]=0
    X=test_df[["pij_t",'log(cij)',"E","abs_p", "avg_p",'abs_c',"avg_c"]]
    y=test_df["ytij"]
    print()
    mods_e.append(logi_mod(X,y, False, i-10,i-1 ))

#########################################
1950 to 1959 inclusive
#########################################
[0 1]

(767, 6)

Percentage of positive outcomes: 38.46%

Logistic Regression Results:
    Variable  Coefficient  Std. Error   z-value       p-value Significance  \
0  Intercept     1.037568    3.111562  0.333456  7.387904e-01                
1      pij_t     0.268692    7.893370  0.034040  9.728451e-01                
2   log(cij)     0.095716    0.337657  0.283470  7.768165e-01                
3      abs_p     0.032673    0.007007  4.663164  3.113843e-06          ***   
4      avg_p    -0.077822    0.012107 -6.428045  1.292559e-10          ***   
5      abs_c     0.760855    0.329151  2.311565  2.080166e-02            *   
6      avg_c    -1.559469    0.654230 -2.383671  1.714093e-02            *   

    t-value    p_values_t Significance_t  
0  0.333456  7.388823e-01                 
1  0.034040  9.728540e-01                 
2  0.283470  7.768936e-01                 
3  4.6631

#########################################
1960 to 1969 inclusive
#########################################
[0 1]

(1439, 6)

Percentage of positive outcomes: 39.40%

Logistic Regression Results:
    Variable  Coefficient  Std. Error    z-value       p-value Significance  \
0  Intercept     1.274863    4.642626   0.274599  7.836240e-01                
1      pij_t     0.071574    9.824579   0.007285  9.941873e-01                
2   log(cij)     0.075171    0.503936   0.149169  8.814205e-01                
3      abs_p     0.055111    0.007523   7.326101  2.369216e-13          ***   
4      avg_p    -0.144346    0.013630 -10.590230  0.000000e+00          ***   
5      abs_c     0.070052    0.272762   0.256826  7.973128e-01                
6      avg_c    -0.696265    0.474717  -1.466695  1.424590e-01                

     t-value    p_values_t Significance_t  
0   0.274599  7.836635e-01                 
1   0.007285  9.941883e-01                 
2   0.149169  8.814415e-01              

In [519]:
upd_w_lags_10[(upd_w_lags_10["pij_t"]==0)&(upd_w_lags_10["shortest_path_count"]==0)&(upd_w_lags_10["avg_p"]==0)&(upd_w_lags_10["avg_c"]==0)&(upd_w_lags_10["tij"]<=upd_w_lags_10["tij_1"])].shape
#[["shortest_path_count","pij_t","tij","tij_1"]].head()

(5733, 26)